# Reforumlated Markowitz Model - Optimization Codes 

Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp  
import cvxopt
import random

Importing the Data

In [2]:
df = pd.read_excel("Daily_Returns.xlsx")

Adding the date as an index

In [3]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Renaming the DataFrame

In [4]:
df_returns=df

Calculating the expected returns and the variance covariance matrix

In [5]:
# Calculate expected returns for each asset
expected_returns = df_returns.mean()

# Calculate the variance-covariance matrix
cov_matrix = df_returns.cov()


#Converting the expected returns to a numpy array
expected_returns = expected_returns.values

#The number of assets
n_assets = len(expected_returns)


Setting the rest of the constraints for the reforumated problem

In [22]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 0.1

# Maximum units of each asset that can be purchased (D)
D = 5

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 15

# The number of shares the investor would like to purchase
S = 20

In [23]:
# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase
y = cp.Variable(n_assets, boolean=True)  # Binary variable for asset inclusion

In [24]:
# Creating the weights constraint
w = x/S

Setting up the optimization problem

In [25]:

# Constraints
risk_constraint = cp.quad_form(w, cov_matrix) <=  R**2  # Risk constraint
diversification_constraints = [x[i] <= D*y[i] for i in range(n_assets)]  # Diversification and linking
cardinality_constraint = cp.sum(y) <= K  # Cardinality constraint
non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 
total_sum_constraint = [cp.sum(x)==S]

# Objective function to maximize total expected return
objective = cp.Maximize(expected_returns @ w)

# Problem definition
problem = cp.Problem(objective, [risk_constraint, cardinality_constraint] + non_negativity_constraints +diversification_constraints + total_sum_constraint )

Solving the optimization problem

In [26]:
# Solving the problem
problem.solve(solver=cp.CPLEX)


# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)
print("Assets included in the portfolio:", y.value)

Status: optimal
Maximum Expected Return: 0.0012809926490892003
Units of each asset to purchase: [-0.  5. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  5. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0.  5. -0. -0. -0. -0. -0. -0. -0.  5. -0. -0. -0. -0. -0.
 -0.]
Assets included in the portfolio: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

Now let us test to see if all the constraints are satisfied

In [33]:
# Convert x.value and y.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
y_optimized = np.array(y.value)
w_optimized = np.array(x_optimized/S)

# Constraint 1: Risk constraint
risk_value = np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized)
print(f"Risk Constraint Satisfied: {np.sqrt(risk_value) <= R}")
print(f"Risk Value: {np.sqrt(risk_value)} <= {R}")

# Constraint 2: Diversification and linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Diversification and Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}") 

# Constraint 5: sum of shares constraint
total_share = np.sum(x_optimized)
print(f"Sum of units Constraint Satisfied: {total_share <= S}")
print(f"Number of Units Included: {total_share} <= {S}")

Risk Constraint Satisfied: True
Risk Value: 0.019487885441359105 <= 0.1
Diversification and Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 4 <= 15
Non-negativity Constraints Satisfied: True
Sum of units Constraint Satisfied: True
Number of Units Included: 20.0 <= 20


Checking the assets to invest in

In [34]:
values_list = x.value

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Column Name': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Column Name'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)


  Column Name  Value
0        AAPL    5.0
1        ALGN    5.0
2        NFLX    5.0
3        TSLA    5.0


Looping it all